In [1]:
!pip install spacy
!python -m spacy download fr_core_news_sm
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 612 kB 2.1 MB/s eta 0:00:01
  Created wheel for regex: filename=regex-2018.1.10-cp36-cp36m-linux_x86_64.whl size=552149 sha256=991fa1e6fd17a2839471a7a3d0ed620a765757b4f98caf62d3c5b92395ba40b8
  Stored in directory: /home/work/.cache/pip/wheels/4b/72/79/772fa2486e5eafc62bd47cd83bf724c129569dd09c24d55cc0
Successfully built regex
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 39.8 MB 8.2 MB/s eta 0:00:01
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.0.0-py3-none-any.whl size=39841462 sha256=607f217230422231e18421e9c09b455e1db038dc64585040fef14bb86d26d29a
  Stored in directory: /tmp/pip-ephem-wheel-cache-c3ivqfma/wheels/46/df/13/b060ba26a65a7cb17e5e7855de0251724fc2862d7843e8af01
Successfully built fr-core-news-sm

    Error: Couldn't link model to 'fr_core_news_sm'
    Creating a symlin

In [92]:
import spacy
import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset

spacy_en = spacy.load('en_core_web_sm')
spacy_fr = spacy.load('fr_core_news_sm')

def tokenize_en(sentence):
    return [tok.text for tok in spacy_en.tokenizer(sentence)]

# REVERSE SENTENCES
def tokenize_fr(sentence):
    return [tok.text for tok in spacy_fr.tokenizer(sentence)]

TRG = Field(tokenize=tokenize_en, init_token = "<sos>", eos_token = "<eos>")
SRC = Field(tokenize=tokenize_fr, init_token = "<sos>", eos_token = "<eos>")

In [2]:
from datetime import datetime
print(datetime.now().strftime('%Y-%m-%d-%H:%M:%S'))

# associate the text in the 'English' trg column with the TRG field, # and 'French' src column with SRC field
data_fields = [('trg', TRG), ('src', SRC)]

train_data, test_data = TabularDataset.splits(path='.',
                                  train = 'train_data_20.csv',
                                  test = 'test_data_20.csv',
                                  format = 'csv',
                                  fields = data_fields,
                                  skip_header = True )

print(datetime.now().strftime('%Y-%m-%d-%H:%M:%S'))

2021-04-12-11:20:23
2021-04-12-11:25:45


In [3]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

훈련 샘플의 개수 : 535428
테스트 샘플의 개수 : 133857


In [4]:
TRG.build_vocab(train_data, min_freq=2)
SRC.build_vocab(train_data, min_freq=2)

In [5]:
print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 42405
len(TRG): 32929


In [6]:
print(TRG.vocab.stoi['the'])
print(TRG.vocab.itos[4])

5
.


In [7]:
print(TRG.vocab.stoi["abcabc"]) # 없는 단어: 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi["<sos>"]) # <sos>: 2
print(TRG.vocab.stoi["<eos>"]) # <eos>: 3
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
0
222


In [8]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

BATCH_SIZE = 8

train_iterator, valid_iterator = BucketIterator.splits(
    (train_data, test_data),
    sort = False, # Tabular -> error. 
    batch_size = BATCH_SIZE,
    device = device)

cuda


In [9]:
for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg

    print(f"첫 번째 배치 크기: {src.shape}")

    # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
    for i in range(src.shape[0]):
        print(f"인덱스 {i}: {src[i][0].item()}")

    # 첫 번째 배치만 확인
    break

첫 번째 배치 크기: torch.Size([25, 8])
인덱스 0: 2
인덱스 1: 136
인덱스 2: 12
인덱스 3: 969
인덱스 4: 1548
인덱스 5: 27
인덱스 6: 3455
인덱스 7: 21
인덱스 8: 4
인덱스 9: 3
인덱스 10: 1
인덱스 11: 1
인덱스 12: 1
인덱스 13: 1
인덱스 14: 1
인덱스 15: 1
인덱스 16: 1
인덱스 17: 1
인덱스 18: 1
인덱스 19: 1
인덱스 20: 1
인덱스 21: 1
인덱스 22: 1
인덱스 23: 1
인덱스 24: 1


## Encoder
* hyperparameter
    * input_dim : 하나의 단어에 대한 one-hot encoding 차원
    * embed_dim : embedding 차원
    * enc_hidden_dim : encoder hidden state dim
    * dec_hidden_dim : decoder hidden state dim
    * dropout_ratio : dropout !pip install spacy


In [10]:
import torch.nn as nn

# 인코더(Encoder) 아키텍처 정의
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, enc_hidden_dim, dec_hidden_dim, dropout_ratio):
        super().__init__()

        # 임베딩(embedding)은 원-핫 인코딩(one-hot encoding)을 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(input_dim, embed_dim)

        # 양방향(bidirectional) GRU 레이어
        self.rnn = nn.GRU(embed_dim, enc_hidden_dim, bidirectional=True)

        # FC 레이어
        self.fc = nn.Linear(enc_hidden_dim * 2, dec_hidden_dim)

        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더는 소스 문장을 입력으로 받아 문맥 벡터(context vector)를 반환        
    def forward(self, src):
        # src: [단어 개수, 배치 크기]: 각 단어의 인덱스(index) 정보
        embedded = self.dropout(self.embedding(src))
        # embedded: [단어 개수, 배치 크기, 임베딩 차원]

        outputs, hidden = self.rnn(embedded)
        # outputs: [단어 개수, 배치 크기, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보
        # hidden: [레이어 개수 * 방향의 수, 배치 크기, 인코더 히든 차원]: 현재까지의 모든 단어의 정보

        # hidden은 [forward_1, backward_1, forward_2, backward_2, ...] 형태로 구성
        # 따라서 hidden[-2, :, :]은 forwards의 마지막 값
        # 따라서 hidden[-1, :, :]은 backwards의 마지막 값
        # 디코더(decoder)의 첫 번째 hidden (context) vector는 인코더의 마지막 hidden을 이용
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)))

        # outputs은 Attention 목적으로, hidden은 context vector 목적으로 사용
        return outputs, hidden

## Attention architecture
* 하나의 attention은 인코더의 전체 토큰에 대한 출력을 입력으로 받는 FC파라미터를 공유해서 사용
* (전체 인코더 출력 + 현재 디코더의 히든) $\to$ 디코더의 히든 $\to$ 실제 attention 값
* hyperparameter
    * enc_hidden_dim : 인코더의 hidden state 차원
    * dec_hidden_dim : 디코더의 hidden state 차원 

In [11]:
import torch.nn.functional as F

# 어텐션(Attention) 아키텍처 정의
class Attention(nn.Module):
    def __init__(self, enc_hidden_dim, dec_hidden_dim):
        super().__init__()

        self.attn = nn.Linear((enc_hidden_dim * 2) + dec_hidden_dim, dec_hidden_dim)
        self.v = nn.Linear(dec_hidden_dim, 1, bias=False)

    def forward(self, hidden, enc_outputs):
        # hidden: [배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
        # enc_outputs: [단어 개수, 배치 크기, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보
        batch_size = enc_outputs.shape[1]
        src_len = enc_outputs.shape[0]

        # 현재 디코더의 히든 상태(hidden state)를 src_len만큼 반복
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        enc_outputs = enc_outputs.permute(1, 0, 2)
        # hidden: [배치 크기, 단어 개수, 디코더 히든 차원]: 현재까지의 모든 단어의 정보
        # enc_outputs: [배치 크기, 단어 개수, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보

        energy = torch.tanh(self.attn(torch.cat((hidden, enc_outputs), dim=2)))
        # energy: [배치 크기, 단어 개수, 디코더 히든 차원]

        attention = self.v(energy).squeeze(2)
        # attention: [배치 크기, 단어 개수]: 실제 각 단어에 대한 어텐선(attention) 값들

        return F.softmax(attention, dim=1)

## Decoder architecture
* 주어진 context vector를 타겟 문장으로 디코딩함
* 단, 디코딩 과정에서 매번 인코더의 모든 출력에 대하여 attention 함
* hyperparameter
    * output_dim : 하나의 단어에 대한 원핫 인코딩 차원
    * embed_dim : 임베딩 차원
    * enc_hidden_dim : 인코더의 hidden state 차원
    * dec_hidden_dim : 디코더의 hidden state 차원 
    * dropout_ratio : dropout proportion

In [12]:

# 디코더(Decoder) 아키텍처 정의
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, enc_hidden_dim, dec_hidden_dim, dropout_ratio, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention

        # 임베딩(embedding)은 원-핫 인코딩(one-hot encoding) 말고 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(output_dim, embed_dim)

        # GRU 레이어
        self.rnn = nn.GRU((enc_hidden_dim * 2) + embed_dim, dec_hidden_dim)

        # FC 레이어
        self.fc_out = nn.Linear((enc_hidden_dim * 2) + dec_hidden_dim + embed_dim, output_dim)
        
        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

    # 디코더는 현재까지 출력된 문장에 대한 정보를 입력으로 받아 타겟 문장을 반환     
    def forward(self, input, hidden, enc_outputs):
        # input: [배치 크기]: 단어의 개수는 항상 1개이도록 구현
        # hidden: [배치 크기, 히든 차원]
        # enc_outputs: [단어 개수, 배치 크기, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보
        input = input.unsqueeze(0)
        # input: [단어 개수 = 1, 배치 크기]

        embedded = self.dropout(self.embedding(input))
        # embedded: [단어 개수 = 1, 배치 크기, 임베딩 차원]

        attention = self.attention(hidden, enc_outputs)
        # attention: [배치 크기, 단어 개수]: 실제 각 단어에 대한 어텐선(attention) 값들
        attention = attention.unsqueeze(1)
        # attention: [배치 크기, 1, 단어 개수]: 실제 각 단어에 대한 어텐선(attention) 값들

        enc_outputs = enc_outputs.permute(1, 0, 2)
        # enc_outputs: [배치 크기, 단어 개수, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보

        weighted = torch.bmm(attention, enc_outputs) # 행렬 곱 함수
        # weighted: [배치 크기, 1, 인코더 히든 차원 * 방향의 수]

        weighted = weighted.permute(1, 0, 2)
        # weighted: [1, 배치 크기, 인코더 히든 차원 * 방향의 수]
        
        rnn_input = torch.cat((embedded, weighted), dim=2)
        # rnn_input: [1, 배치 크기, 인코더 히든 차원 * 방향의 수 + embed_dim]: 어텐션이 적용된 현재 단어 입력 정보
        
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        # output: [단어 개수, 배치 크기, 디코더 히든 차원 * 방향의 수]
        # hidden: [레이어 개수 * 방향의 수, 배치 크기, 디코더 히든 차원]: 현재까지의 모든 단어의 정보

        # 현재 예제에서는 단어 개수, 레이어 개수, 방향의 수 모두 1의 값을 가짐
        # 따라서 output: [1, 배치 크기, 디코더 히든 차원], hidden: [1, 배치 크기, 디코더 히든 차원]
        # 다시 말해 output과 hidden의 값 또한 동일
        assert (output == hidden).all()

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))
        # prediction = [배치 크기, 출력 차원]
        
        # (현재 출력 단어, 현재까지의 모든 단어의 정보)
        return prediction, hidden.squeeze(0)

## Seq2Seq with Attention
* encoder-decoder architecture
    * encoder : sequence(src) $\to$ context vector
    * decoder : context vector $\to$ sequence(trg)
        * 디코더는 한 단어 당 한 번씩 결과를 구함
        * 인코더의 모든 출력을 참고하여 어텐션(attention)을 진행 
    * teacher forcing : 디코더의 예측(prediction)을 다음 입력으로 사용하지 않고, 실제 목표 출력(ground-truth)을 다음 입력으로 사용

In [13]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    # 학습할 때는 완전한 형태의 소스 문장, 타겟 문장, teacher_forcing_ratio를 넣기
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [단어 개수, 배치 크기]
        # trg: [단어 개수, 배치 크기]
        # 먼저 인코더를 거쳐 전체 출력과 문맥 벡터(context vector)를 추출
        enc_outputs, hidden = self.encoder(src)

        # 디코더(decoder)의 최종 결과를 담을 텐서 객체 만들기
        trg_len = trg.shape[0] # 단어 개수
        batch_size = trg.shape[1] # 배치 크기
        trg_vocab_size = self.decoder.output_dim # 출력 차원
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # 첫 번째 입력은 항상 <sos> 토큰
        input = trg[0, :]

        # 타겟 단어의 개수만큼 반복하여 디코더에 포워딩(forwarding)
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden, enc_outputs)

            outputs[t] = output # FC를 거쳐서 나온 현재의 출력 단어 정보
            top1 = output.argmax(1) # 가장 확률이 높은 단어의 인덱스 추출

            # teacher_forcing_ratio: 학습할 때 실제 목표 출력(ground-truth)을 사용하는 비율
            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[t] if teacher_force else top1 # 현재의 출력 결과를 다음 입력에서 넣기

        return outputs

In [14]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENCODER_EMBED_DIM = 128
DECODER_EMBED_DIM = 128
ENCODER_HIDDEN_DIM = 256
DECODER_HIDDEN_DIM = 256
ENC_DROPOUT_RATIO = 0.5
DEC_DROPOUT_RATIO = 0.5

In [15]:
# 어텐션(attention) 객체 선언
attn = Attention(ENCODER_HIDDEN_DIM, DECODER_HIDDEN_DIM)

# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, ENCODER_HIDDEN_DIM, DECODER_HIDDEN_DIM, ENC_DROPOUT_RATIO)
dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, ENCODER_HIDDEN_DIM, DECODER_HIDDEN_DIM, DEC_DROPOUT_RATIO, attn)

# Seq2Seq 객체 선언
model = Seq2Seq(enc, dec, device).to(device)


In [16]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(42405, 128)
    (rnn): GRU(128, 256, bidirectional=True)
    (fc): Linear(in_features=512, out_features=256, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=768, out_features=256, bias=True)
      (v): Linear(in_features=256, out_features=1, bias=False)
    )
    (embedding): Embedding(32929, 128)
    (rnn): GRU(640, 256)
    (fc_out): Linear(in_features=896, out_features=32929, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [17]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [18]:
# 모델 학습(train) 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0
    
    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()

        output = model(src, trg)
        # output: [출력 단어 개수, 배치 크기, 출력 차원]
        output_dim = output.shape[-1]
        
        # 출력 단어의 인덱스 0은 사용하지 않음
        output = output[1:].view(-1, output_dim)
        # output = [(출력 단어의 개수 - 1) * batch size, output dim]
        trg = trg[1:].view(-1)
        # trg = [(타겟 단어의 개수 - 1) * batch size]
        
        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산
        
        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        # 파라미터 업데이트
        optimizer.step()
        
        # 전체 손실 값 계산
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [19]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0
    
    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # 평가할 때 teacher forcing는 사용하지 않음
            output = model(src, trg, 0)
            # output: [출력 단어 개수, 배치 크기, 출력 차원]
            output_dim = output.shape[-1]
            
            # 출력 단어의 인덱스 0은 사용하지 않음
            output = output[1:].view(-1, output_dim)
            # output = [(출력 단어의 개수 - 1) * batch size, output dim]
            trg = trg[1:].view(-1)
            # trg = [(타겟 단어의 개수 - 1) * batch size]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [20]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [21]:
import time
import math
import random
from tqdm.auto import tqdm

N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

for epoch in tqdm(range(N_EPOCHS)):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best_seq2seq_attn_20.pt')
        print("best_seq2seq_attn save")

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')
    
    torch.save(model.state_dict(), 'seq2seq_attn_20.pt')
    print("\t",epoch+1,"th model save")

best_seq2seq_attn save
Epoch: 01 | Time: 138m 57s
	Train Loss: 4.938 | Train PPL: 139.444
	Validation Loss: 4.833 | Validation PPL: 125.552
	 1 th model save
best_seq2seq_attn save
Epoch: 02 | Time: 139m 7s
	Train Loss: 3.941 | Train PPL: 51.486
	Validation Loss: 4.401 | Validation PPL: 81.559
	 2 th model save
best_seq2seq_attn save
Epoch: 03 | Time: 139m 26s
	Train Loss: 3.513 | Train PPL: 33.555
	Validation Loss: 4.239 | Validation PPL: 69.365
	 3 th model save
best_seq2seq_attn save
Epoch: 04 | Time: 139m 14s
	Train Loss: 3.285 | Train PPL: 26.705
	Validation Loss: 4.143 | Validation PPL: 62.979
	 4 th model save
best_seq2seq_attn save
Epoch: 05 | Time: 138m 57s
	Train Loss: 3.144 | Train PPL: 23.204
	Validation Loss: 4.098 | Validation PPL: 60.201
	 5 th model save
best_seq2seq_attn save
Epoch: 06 | Time: 138m 59s
	Train Loss: 3.046 | Train PPL: 21.028
	Validation Loss: 4.092 | Validation PPL: 59.880
	 6 th model save
best_seq2seq_attn save
Epoch: 07 | Time: 138m 58s
	Train Loss: 

### ADDITIONAL TRAINNING

In [ ]:
# TRANSFER LERANING
model.load_state_dict(torch.load('best_seq2seq_attn_20.pt'))

In [26]:
# Add lr /10
optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [ ]:
import time
import math
import random
from tqdm.auto import tqdm

N_EPOCHS = 5
CLIP = 1
best_valid_loss = float('inf')

for epoch in tqdm(range(N_EPOCHS)):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best_seq2seq_attn_20.pt')
        print("best_seq2seq_attn_20 save")

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')
    
    torch.save(model.state_dict(), 'seq2seq_attn_20.pt')
    print(epoch,"th model save")

0 th model save
best_seq2seq save
Epoch: 02 | Time: 356m 4s
	Train Loss: 2.867 | Train PPL: 17.591
	Validation Loss: 4.416 | Validation PPL: 82.783
1 th model save
best_seq2seq save
Epoch: 03 | Time: 356m 54s
	Train Loss: 2.862 | Train PPL: 17.501
	Validation Loss: 4.406 | Validation PPL: 81.961
2 th model save
best_seq2seq save
Epoch: 04 | Time: 358m 58s
	Train Loss: 2.858 | Train PPL: 17.419
	Validation Loss: 4.402 | Validation PPL: 81.637
3 th model save


In [22]:
# 번역(translation) 함수
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):
    model.eval() # 평가 모드

    if isinstance(sentence, str):
        nlp = spacy.load('fr_core_news_sm')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    # print(f"전체 소스 토큰: {tokens}")

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    # print(f"소스 문장 인덱스: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # 인코더(endocer)에 소스 문장을 넣어 문맥 벡터(context vector) 계산
    with torch.no_grad():
        enc_outputs, hidden = model.encoder(src_tensor)

    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        # 이전에 출력한 단어가 현재 단어로 입력될 수 있도록
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden, enc_outputs)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token) # 출력 문장에 더하기

        # <eos>를 만나는 순간 끝
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # 첫 번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:]

In [45]:
src = tokenize_fr("Reprise de la session.")

print(f'소스 문장: {src}')
print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['Reprise', 'de', 'la', 'session', '.']
모델 출력 결과: The the session . <eos>


In [36]:
src = " ".join(['Mais', "l'", 'alliance', 'occidentale', 'ne', 'combat', 'pas', 'dans', 'les', 'Balkans', 'aux', 'seuls', 'motifs', 'de', 'mettre', 'un', 'terme', 'à', 'la', 'souffrance', 'et', 'de', 'défendre', 'le', 'droit', 'et', 'la', 'justice', '.'])
src

"Mais l' alliance occidentale ne combat pas dans les Balkans aux seuls motifs de mettre un terme à la souffrance et de défendre le droit et la justice ."

In [39]:
pred_30 = " ".join(['The', 'Western', 'alliance', 'is', 'not', 'fighting', 'in', 'the', '<unk>', 'to', 'the', 'the', 'reasons', 'of', 'putting', 'the', 'end', 'to', 'defend', 'and', 'and', 'justice', 'and', 'justice', '.'])
trg

'The western alliance is not just fighting in the Balkans to prevent suffering and to secure justice and the rule of law in the Balkans .'

In [40]:
src_token = tokenize_fr(src)



print(f'소스 문장: {src}')
print("20 모델 출력 결과:", " ".join(translate_sentence(src_token, SRC, TRG, model, device)))
print("30 모델 출력 결과:", pred_30)

소스 문장: Mais l' alliance occidentale ne combat pas dans les Balkans aux seuls motifs de mettre un terme à la souffrance et de défendre le droit et la justice .
20 모델 출력 결과: But the Western alliance does not stop the the only grounds for the and to defend and law . <eos>
30 모델 출력 결과: The Western alliance is not fighting in the <unk> to the the reasons of putting the end to defend and and justice and justice .


In [46]:
!pip install sacrebleu

Defaulting to user installation because normal site-packages is not writeable


In [47]:
!sacrebleu --list

usage: sacrebleu [-h]
                 [--test-set {wmt18,wmt18/test-ts,wmt18/dev,wmt17,wmt17/B,wmt17/tworefs,wmt17/improved,wmt17/dev,wmt17/ms,wmt16,wmt16/B,wmt16/tworefs,wmt16/dev,wmt15,wmt14,wmt14/full,wmt13,wmt12,wmt11,wmt10,wmt09,wmt08,wmt08/nc,wmt08/europarl,iwslt17,iwslt17/tst2016,iwslt17/tst2015,iwslt17/tst2014,iwslt17/tst2013,iwslt17/tst2012,iwslt17/tst2011,iwslt17/tst2010,iwslt17/dev2010}]
                 [-lc] [--smooth {exp,floor,none}]
                 [--tokenize {13a,intl,zh,none}] [--language-pair LANGPAIR]
                 [--download DOWNLOAD] [--echo {src,ref,both}] [--input INPUT]
                 [--metrics {bleu,chrf} [{bleu,chrf} ...]]
                 [--chrf-order CHRF_ORDER] [--chrf-beta CHRF_BETA]
                 [--chrf-whitespace] [--short] [--score-only] [--force]
                 [--quiet] [--encoding ENCODING] [--citation] [-V]
                 [refs [refs ...]]
sacrebleu: error: unrecognized arguments: --list


In [90]:
import sacrebleu

def calculate_sacre_bleu(data, src_field, trg_field, model, device):
    
    trgs = []
    pred_trgs = []
    total_bleu = 0
    n = 1000
    
    for i in tqdm(range(n)):
        
        src = vars(data[i])['src']
        trg = " ".join(vars(data[i])['trg'])
        
        pred_trg = translate_sentence(src, src_field, trg_field, model, device)
        
        #cut off <eos> token
        pred_trg = " ".join(pred_trg[:-1])
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
        bleu = sacrebleu.corpus_bleu(trg, pred_trg)
        total_bleu += bleu.score
        
        if i % 100 == 0 :
            print("src :"," ".join(src))
            print("trg :",trg)
            print("pred :",pred_trg)
            print("score:",bleu.score)
            print("\n")
    avg = total_bleu / i        
        
        
    return avg

In [91]:
bleu_score = calculate_sacre_bleu(test_data, SRC, TRG, model, device)

print(f'BLEU score = {bleu_score:.2f}')

src : C' est pourquoi une importance particulière doit être accordée au principe de subsidiarité .
trg : For this reason , particular significance must be attached to the principle of subsidiarity .
pred : That is why special importance must be given to the principle of subsidiarity .
score: 36.41410562218428


src : Les gouvernements ont le devoir d' y répondre en complétant les règles existantes par des protections alternatives à l' asile traditionnel .
trg : Governments have a duty to deal with this by supplementing existing legislation with means of protection other than conventional asylum .
pred : Governments have the duty to respond to the existing rules governing alternative by by traditional . .
score: 5.908002399935303


src : Le Parlement européen est tout à fait compétent pour contrôler l' ensemble des activités de la Commission européenne , jour après jour .
trg : The European Parliament has the ability to monitor all the European Commission 's activities on a daily basis 